In [ ]:
!pip install streamlit
!npm install localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4 MB/s eta 0:00:00
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.588s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit f

In [ ]:

import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [ ]:
# Necessary library installations
!pip install -q -U transformers==4.37.2
!pip install -q bitsandbytes==0.41.3 accelerate==0.25.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 30.4 MB/s eta 0:00:00


In [ ]:
%%writefile app.py

import streamlit as st
from PIL import Image
import requests
import torch
from transformers import BitsAndBytesConfig, pipeline

# Since we cannot use !pip in a Streamlit script, ensure the necessary packages are installed beforehand:
# pip install streamlit transformers==4.37.2 bitsandbytes-cuda111==0.41.3 accelerate==0.25.0

def configure_quantization():
    """Configure the model for 4-bit precision using quantization."""
    return BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16
    )

@st.cache_resource
def initialize_pipeline(model_id, quantization_config):
    """Initialize the image-to-text pipeline with quantization."""
    return pipeline("image-to-text", model=model_id, model_kwargs={"quantization_config": quantization_config})

def analyze_image_with_text(pipe, image, prompt, max_tokens=2):
    """Analyze an image with accompanying text and return the model's output."""
    result = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": max_tokens})
    print(result)
    return result[0]["generated_text"]

# Streamlit UI
st.title('Multimodal Sentiment Analysis with bakLlava')

# Text input
user_input_text = st.text_area("Enter your text here:")

# Image upload
uploaded_image = st.file_uploader("Upload an image:", type=["jpg", "jpeg", "png"])

if uploaded_image is not None:
    # Display the uploaded image
    image = Image.open(uploaded_image)
    st.image(image, caption='Uploaded Image', use_column_width=True)

    # Model initialization and analysis
    if st.button('Analyze!'):
        with st.spinner('Analyzing...'):
            quantization_config = configure_quantization()
            model_id = "llava-hf/bakLlava-v1-hf"
            llava_pipe = initialize_pipeline(model_id, quantization_config)
            prompt = f"""USER: ******
            CONTEXT: You are an expert review classifier tasked with categorizing reviews on Amazon products, assigning ratings ranging from 1 to 5 stars. Identify the sentiment of the review, and check the image for its condition, like damages, etc. Below are sample reviews to provide context for your classification task:

            Review 1:
            Product - Pixel z
            Review - The Google Pixel 8 presents a sleek design and sturdy construction. However, I encountered performance issues with software tuning, particularly noticeable with apps like YouTube freezing or stuttering. Given Google's control over hardware and software, this underperformance is disappointing. Despite subsequent updates, initial performance falls short of expectations for an 8th generation device. Rating - 3 stars

            Review 2:
            Product - Pixel z
            Review - Transitioning from a Pixel 3 XL to the Pixel 8, I'm delighted with the enhancements and new features. Notably, the integration of AI and the exceptional camera elevate daily use. While the protruding camera design is a minor drawback, overall, the device impresses. Rating - 5 stars

            Review 3:
            Product - Protein Powder
            Review - Regrettably, this protein powder fails to mix well, resulting in a poor texture despite following instructions. While the taste remains satisfactory, the decline in mixability compared to previous versions prompts disappointment, leading me to revert to my previous brand. Rating - 2 stars

            Review 4:
            Product - Protein Shake
            Review - Encountered an unexpected chunky, plastic-like film in the shake, a stark departure from my previous experiences with this product. Presumably, this batch is defective, as such issues were never encountered before. Rating - 1 star

            Review 5:
            Product - Dumbbell
            Review - These dumbbells complement my home workout regimen excellently. Surpassing my expectations in terms of color and quality, they are reasonably priced. I highly recommend them for light workouts. Rating - 5 stars
            TASK:

            User Review - {user_input_text}
            image - <image>
            rate this review give only the number from 1 to 5, lean more on the postive side unless damage is mentioned:
            ASSISTANT:"""

            output_text = analyze_image_with_text(llava_pipe, image, prompt)
            st.write("Analysis Result:")
            #st.write(output_text[0]['generated_text'][-1:])
            last_line = output_text.split('\n')[-1]
            st.write(last_line)

else:
    st.write("Please upload an image.")

Writing app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 35.240.162.130


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 1.525s
your url is: https://flat-oranges-tap.loca.lt


In [ ]:
import requests
from PIL import Image
import torch
from transformers import BitsAndBytesConfig, pipeline


In [ ]:
def load_image_from_url(url):
    """Load an image from a URL for analysis."""
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        return Image.open(response.raw)
    except requests.RequestException as e:
        print(f"Error loading image: {e}")
        return None

In [ ]:
def configure_quantization():
    """Configure the model for 4-bit precision using quantization."""
    return BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16
    )

In [ ]:
def initialize_pipeline(model_id, quantization_config):
    """Initialize the image-to-text pipeline with quantization."""
    return pipeline("image-to-text", model=model_id, model_kwargs={"quantization_config": quantization_config})

In [ ]:
def analyze_image_with_text(pipe, image, prompt, max_tokens=200):
    """Analyze an image with accompanying text and return the model's output."""
    result = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": max_tokens})
    return result[0]["generated_text"]


In [ ]:
# Main execution
image_url = "https://llava-vl.github.io/static/images/view.jpg"
image = load_image_from_url(image_url)

In [ ]:
quantization_config = configure_quantization()
model_id = "llava-hf/llava-1.5-7b-hf"
llava_pipe = initialize_pipeline(model_id, quantization_config)

In [ ]:

prompt = "USER: <image>\nWhat should I be cautious about when visiting this place?\nASSISTANT:"
output_text = analyze_image_with_text(llava_pipe, image, prompt)

print(output_text)